In [1]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive
  

Mounted at /gdrive
/gdrive


In [2]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install vecstack
!pip install mlxtend


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=924c155e83a319345382e68eb515d9dd58362565a033c7bb0161ae21c791cf70
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

# Import libraries for feature selection

#from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import warnings
warnings.filterwarnings("ignore")


In [5]:
#Read files
trainfile = r'/gdrive/My Drive/CIS508-FALL2020/ASSN6/Churn-Train-2.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508-FALL2020/ASSN6/Churn-Test-2.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 
print()
print(test_data.shape)
print(test_data.head()) 


(1165, 20)
  State  Account Length  Area Code Int'l Plan VMail Plan  VMail Message  \
0    KS             128        415         no        yes             25   
1    OH             107        415         no        yes             26   
2    NJ             137        415         no         no              0   
3    OH              84        408        yes         no              0   
4    OK              75        415        yes         no              0   

   Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
0     265.1        110       45.07     197.4         99       16.78   
1     161.6        123       27.47     195.5        103       16.62   
2     243.4        114       41.38     121.2        110       10.30   
3     299.4         71       50.90      61.9         88        5.26   
4     166.7        113       28.34     148.3        122       12.61   

   Night Mins  Night Calls  Night Charge  Intl Mins  Intl Calls  Intl Charge  \
0       244.7           91     

In [6]:
#convert categorical features by one-hot encoding
categoricalFeatures = ["State","Int'l Plan","VMail Plan"]

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([train_data,test_data], keys=[0,1])

#Do one Hot encoding for categorical features
#combined_Data["TARGET"]=combined_Data["TARGET"].map({"Yes":1,"No":0})
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)
print(combined_Data['TARGET'])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)

print(X_train.shape)
print(X_test.shape)

y_train=X_train["TARGET"]
y_test=X_test["TARGET"]


X_train = X_train.drop('TARGET',axis=1)
X_test = X_test.drop('TARGET',axis=1)

export_csv1 = X_train.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/X_train-AfterOneHotEncoding.csv')
export_csv2 = X_test.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/X_test-AfterOneHotEncoding.csv')

print(X_train.shape)
print(X_test.shape)
#print(X_train)


0  0       0
   1       0
   2       0
   3       0
   4       0
          ..
1  1256    1
   1257    0
   1258    0
   1259    0
   1260    0
Name: TARGET, Length: 2426, dtype: int64
(1165, 72)
(1261, 72)
(1165, 71)
(1261, 71)


In [ ]:
#DECISION TREE=======================================================================================================
#====================================================================================================================
#CONSTRUCT DEFAULT DECISION TREE WITHOUT FEATURE SELECTION AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))
#clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - Decision Tree: ",clf_cv_score.mean())


accuracy Score (training) for Decision TreE:0.906423
Confusion Matrix for Decision Tree
[[1022   47]
 [  71  121]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1069
           1       0.72      0.63      0.67       192

    accuracy                           0.91      1261
   macro avg       0.83      0.79      0.81      1261
weighted avg       0.90      0.91      0.90      1261

=== All Accuracy Scores ===
[0.88034188 0.86324786 0.88034188 0.88034188 0.91452991 0.9137931
 0.90517241 0.87931034 0.87068966 0.93103448]


=== Mean Accuracy Score ===
Mean Accuracy Score - Decision Tree:  0.8918803418803419


In [ ]:
#Do feature selection using a classification model - DECISION TREE
print(clf.feature_importances_)

#print sorted list of important features
importances = clf.feature_importances_
indices = np.argsort(importances)[-10:] #the list is in ascending order
print("\n top 10 features = ", indices, "\n")


[0.005914   0.         0.01229617 0.07757635 0.01795939 0.1469229
 0.03975253 0.01318163 0.10865034 0.06521761 0.02310047 0.03087253
 0.07390862 0.0818684  0.01044089 0.11685448 0.         0.
 0.         0.         0.         0.01086753 0.         0.00050811
 0.         0.00051889 0.00053709 0.         0.         0.
 0.         0.         0.         0.         0.001806   0.
 0.         0.         0.         0.00756965 0.         0.01901419
 0.00224732 0.         0.         0.00067078 0.         0.
 0.         0.         0.         0.00743914 0.         0.
 0.         0.         0.         0.         0.         0.00234295
 0.         0.007293   0.         0.         0.         0.00215743
 0.00703024 0.05178686 0.01740149 0.01764856 0.01864445]

 top 10 features =  [11  6 67  9 12  3 13  8 15  5] 



In [ ]:
#Do feature selection using a classification model - DECISION TREE
#model = SelectFromModel(clf, prefit=True)
model = SelectFromModel(clf, prefit=True, max_features=10, threshold=-np.inf)
#model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X_train)
X_new_SelectedFeatures= pd.DataFrame(X_new)
print(X_new_SelectedFeatures)
export_csv= X_new_SelectedFeatures.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/X_new_SelectedFeatures.csv')



          0      1      2      3      4      5     6    7    8    9
0     265.1  45.07  197.4  16.78  244.7  11.01  10.0  3.0  1.0  1.0
1     161.6  27.47  195.5  16.62  254.4  11.45  13.7  3.0  1.0  1.0
2     243.4  41.38  121.2  10.30  162.6   7.32  12.2  5.0  0.0  1.0
3     299.4  50.90   61.9   5.26  196.9   8.86   6.6  7.0  2.0  0.0
4     166.7  28.34  148.3  12.61  186.9   8.41  10.1  3.0  3.0  0.0
...     ...    ...    ...    ...    ...    ...   ...  ...  ...  ...
1160  112.2  19.07  154.8  13.16  166.7   7.50  10.6  2.0  0.0  1.0
1161  170.7  29.02  179.1  15.22  281.9  12.69   8.2  9.0  3.0  0.0
1162  172.5  29.33  142.6  12.12  102.0   4.59  10.9  2.0  0.0  1.0
1163  194.3  33.03  243.2  20.67  322.2  14.50   7.1  5.0  1.0  1.0
1164  187.8  31.93  129.8  11.03  132.3   5.95  12.0  3.0  2.0  1.0

[1165 rows x 10 columns]


In [ ]:
#Do feature selection using a classification model - DECISION TREE
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new.shape)
print ("\n features_df_new= ", features_df_new)
#print(features_df_new.head())

#save file
export_csv= features_df_new.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/features_df_new.csv')


[False False False  True False  True  True False  True  True False  True
  True  True False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False]

 cols =  [ 3  5  6  8  9 11 12 13 15 67] 

(1165, 10)

 features_df_new=        Day Mins  Day Charge  Eve Mins  Eve Charge  Night Mins  Night Charge  \
0        265.1       45.07     197.4       16.78       244.7         11.01   
1        161.6       27.47     195.5       16.62       254.4         11.45   
2        243.4       41.38     121.2       10.30       162.6          7.32   
3        299.4       50.90      61.9        5.26       196.9          8.86   
4        166.7       28.34     148.3       12.61       186.9          8.41   
...        ...         .

In [ ]:
#CONSTRUCT DEFAULT DECISION TREE - WITH FEATURE SELECTION - AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(features_df_new, y_train)
#Get corresponding test set columns
features_df_new_test = X_test.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_test.shape)

clf_predict=clf.predict(features_df_new_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(features_df_new_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))
#clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
clf_cv_score = cross_val_score(clf, features_df_new, y_train, cv=10, scoring="accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - Decision Tree: ",clf_cv_score.mean())


(1261, 10)
accuracy Score (training) for Decision TreE:0.903251
Confusion Matrix for Decision Tree
[[1026   43]
 [  79  113]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      1069
           1       0.72      0.59      0.65       192

    accuracy                           0.90      1261
   macro avg       0.83      0.77      0.80      1261
weighted avg       0.90      0.90      0.90      1261

=== All Accuracy Scores ===
[0.86324786 0.87179487 0.91452991 0.93162393 0.92307692 0.93103448
 0.90517241 0.87931034 0.88793103 0.93965517]


=== Mean Accuracy Score ===
Mean Accuracy Score - Decision Tree:  0.9047376952549367


In [ ]:
#RANDOM FOREST=================================================================================================
#===============================================================================================================
#CONSTRUCT DEFAULT DECISION TREE WITHOUT FEATURE SELECTION AND OBTAIN RESPECTIVE ACCURACY 
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Fores")
print(confusion_matrix(y_test,rfc_predict))

print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
#clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="accuracy")
print("=== All Accuracy Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - Random Fores: ",rfc_cv_score.mean())


accuracy Score (training) for Random Forest:0.907216
Confusion Matrix for Random Fores
[[1068    1]
 [ 116   76]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1069
           1       0.99      0.40      0.57       192

    accuracy                           0.91      1261
   macro avg       0.94      0.70      0.76      1261
weighted avg       0.91      0.91      0.89      1261

=== All Accuracy Scores ===
[0.90598291 0.94017094 0.91452991 0.8974359  0.92307692 0.93103448
 0.93103448 0.93965517 0.94827586 0.9137931 ]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Fores:  0.9244989684644859


In [ ]:
#RANDOM FOREST=================================================================================================
#Do feature selection using a classification model - RANDOM FOREST
print(rfc.feature_importances_)

#print sorted list of important features
importances = rfc.feature_importances_
indices = np.argsort(importances)[-10:] #the list is in ascending order
print("\n top 10 features = ", indices, "\n")


[3.12530370e-02 1.12383164e-02 2.04434264e-02 1.10766156e-01
 4.17176640e-02 1.00383003e-01 6.13129089e-02 3.55242259e-02
 5.81590359e-02 4.11184632e-02 3.68121069e-02 3.80626809e-02
 5.02974563e-02 3.59589827e-02 5.12711766e-02 9.44657348e-02
 1.45025658e-03 9.98442660e-04 0.00000000e+00 2.15026811e-03
 2.34821335e-03 2.62862826e-03 8.38431410e-04 2.20285555e-03
 1.97490210e-03 2.60616978e-03 2.15172993e-03 4.08192459e-05
 5.72518470e-04 1.45777948e-03 1.68025974e-05 2.34140650e-03
 2.46595023e-03 1.36479800e-03 3.38960540e-03 2.53122870e-03
 2.22780870e-03 3.96636572e-03 3.61445811e-04 1.02403916e-03
 1.90230862e-03 4.26682571e-03 1.80411404e-03 1.23822148e-03
 2.65089514e-05 2.45101340e-03 6.79186543e-06 7.13362508e-03
 1.22823602e-03 3.29907549e-03 1.23067028e-03 3.03155576e-03
 1.04164180e-03 1.70782895e-03 6.20939831e-04 3.60243148e-04
 7.80530952e-04 2.11387528e-03 6.34231648e-05 2.99171701e-03
 2.71978050e-03 2.86761575e-03 1.39225984e-03 6.87804873e-04
 2.77533906e-03 2.642231

In [ ]:
#Do feature selection using a classification model - RANDOM FOREST
#model = SelectFromModel(clf, prefit=True)
model = SelectFromModel(rfc, prefit=True, max_features=10, threshold=-np.inf)
#model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X_train)
X_new_SelectedFeatures_RF= pd.DataFrame(X_new)
print(X_new_SelectedFeatures_RF)
export_csv= X_new_SelectedFeatures_RF.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/X_new_SelectedFeatures_RF.csv')


          0      1      2      3      4      5      6     7     8    9
0     265.1  110.0  45.07  197.4  16.78  244.7  11.01  10.0  2.70  1.0
1     161.6  123.0  27.47  195.5  16.62  254.4  11.45  13.7  3.70  1.0
2     243.4  114.0  41.38  121.2  10.30  162.6   7.32  12.2  3.29  0.0
3     299.4   71.0  50.90   61.9   5.26  196.9   8.86   6.6  1.78  2.0
4     166.7  113.0  28.34  148.3  12.61  186.9   8.41  10.1  2.73  3.0
...     ...    ...    ...    ...    ...    ...    ...   ...   ...  ...
1160  112.2   70.0  19.07  154.8  13.16  166.7   7.50  10.6  2.86  0.0
1161  170.7   55.0  29.02  179.1  15.22  281.9  12.69   8.2  2.21  3.0
1162  172.5   78.0  29.33  142.6  12.12  102.0   4.59  10.9  2.94  0.0
1163  194.3  107.0  33.03  243.2  20.67  322.2  14.50   7.1  1.92  1.0
1164  187.8  117.0  31.93  129.8  11.03  132.3   5.95  12.0  3.24  2.0

[1165 rows x 10 columns]


In [ ]:
#Do feature selection using a classification model - RANDOM FOREST
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_RF = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_RF.shape)
print ("\n features_df_new= ", features_df_new_RF)
#print(features_df_new.head())

#save file
export_csv= features_df_new_RF.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/features_df_new_RF.csv')


[False False False  True  True  True  True False  True  True False  True
  True False  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False]

 cols =  [ 3  4  5  6  8  9 11 12 14 15] 

(1165, 10)

 features_df_new=        Day Mins  Day Calls  Day Charge  Eve Mins  Eve Charge  Night Mins  \
0        265.1        110       45.07     197.4       16.78       244.7   
1        161.6        123       27.47     195.5       16.62       254.4   
2        243.4        114       41.38     121.2       10.30       162.6   
3        299.4         71       50.90      61.9        5.26       196.9   
4        166.7        113       28.34     148.3       12.61       186.9   
...        ...        ...         ...     

In [ ]:
#CONSTRUCT DEFAULT RANDOM FOREST - WITH FEATURE SELECTION - AND OBTAIN RESPECTIVE ACCURACY 
rfc = RandomForestClassifier()
rfc.fit(features_df_new_RF, y_train)
#Get corresponding test set columns
features_df_new_test_RF = X_test.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_test_RF.shape)

rfc_predict=rfc.predict(features_df_new_test_RF)
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(features_df_new_test_RF,y_test)))
print("Confusion Matrix for Random Forest")
print(confusion_matrix(y_test,rfc_predict))

print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
#clf_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
clf_cv_score = cross_val_score(rfc, features_df_new_RF, y_train, cv=10, scoring="accuracy")
print("=== All Accuracy Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - Random Forest: ",rfc_cv_score.mean())


(1261, 10)
accuracy Score (training) for Random Forest:0.893735
Confusion Matrix for Random Forest
[[1053   16]
 [ 118   74]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      1069
           1       0.82      0.39      0.52       192

    accuracy                           0.89      1261
   macro avg       0.86      0.69      0.73      1261
weighted avg       0.89      0.89      0.88      1261

=== All Accuracy Scores ===
[0.90598291 0.94017094 0.91452991 0.8974359  0.92307692 0.93103448
 0.93103448 0.93965517 0.94827586 0.9137931 ]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Forest:  0.9244989684644859


In [7]:
#Sequential FORWARD Search =====================================================================
#===========================================================================================================
#http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

#!pip install mlxtend
!pip install clodsa==1.2.43
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

clf = DecisionTreeClassifier()

sfs1 = SFS(clf, 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=3)

sfs1 = sfs1.fit(X_train,y_train)
sfs1.subsets_



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 6.6 MB/s 
     |████████████████████████████████| 276 kB 62.8 MB/s 
  Created wheel for clodsa: filename=clodsa-1.2.43-py2.py3-none-any.whl size=72157 sha256=1ec5c1cafbde1c4e64a05519d5b912aa4596d4f6fedeff664913c9ccd6d5149d
  Stored in directory: /root/.cache/pip/wheels/2a/b4/7f/e6623a990b7ddebf74b86be2985f9435c7223f8ac0ea2dc52f
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12092 sha256=3d0525a02a5ee980b698bf6da5ca4c6276d82f9c38c01202072a0679d6ec2ebe
  Stored in directory: /root/.cache/pip/wheels/eb/bb/07/25a7f0718ee3fe137384011b8e56070f91cf950ee6047c287f
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62527 sha256=23edd1b7a845ca0d4441ab4354e3f847c56fdb7f1597686fb54b4268f315481b
  Stored in directory: /root/.cache/pip/wheels/92/e3/af/1dc0fdca93232d700ac176af6554cf22b85f3

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:    0.4s finished

[2022-11-02 00:23:36] Features: 1/10 -- score: 0.8832608945308703[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.4s finished

[2022-11-02 00:23:36] Features: 2/10 -- score: 0.8849791075892895[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:    0.4s finished

[2022-11-02 00:23:36] Features: 3/10 -- score: 0.8866973206477088[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

{1: {'feature_idx': (15,),
  'cv_scores': array([0.88431877, 0.87886598, 0.88659794]),
  'avg_score': 0.8832608945308703,
  'feature_names': ('CustServ Calls',)},
 2: {'feature_idx': (15, 37),
  'cv_scores': array([0.88431877, 0.8814433 , 0.88917526]),
  'avg_score': 0.8849791075892895,
  'feature_names': ('CustServ Calls', 'State_ME')},
 3: {'feature_idx': (15, 37, 60),
  'cv_scores': array([0.88431877, 0.88402062, 0.89175258]),
  'avg_score': 0.8866973206477088,
  'feature_names': ('CustServ Calls', 'State_ME', 'State_UT')},
 4: {'feature_idx': (15, 17, 37, 60),
  'cv_scores': array([0.88431877, 0.88402062, 0.89175258]),
  'avg_score': 0.8866973206477088,
  'feature_names': ('CustServ Calls', 'State_AL', 'State_ME', 'State_UT')},
 5: {'feature_idx': (15, 17, 18, 37, 60),
  'cv_scores': array([0.88431877, 0.88402062, 0.89175258]),
  'avg_score': 0.8866973206477088,
  'feature_names': ('CustServ Calls',
   'State_AL',
   'State_AR',
   'State_ME',
   'State_UT')},
 6: {'feature_idx': (

In [8]:
#Sequential forward search result
print(sfs1.k_feature_names_)
print(sfs1.k_score_)


('CustServ Calls', 'State_AL', 'State_AR', 'State_AZ', 'State_CA', 'State_DC', 'State_DE', 'State_HI', 'State_ME', 'State_UT')
0.8866973206477088


In [9]:
#Sequential BACKWARD Search =====================================================================
#===========================================================================================================
#http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

clf = DecisionTreeClassifier()

sbs = SFS(clf, 
           k_features=10,  
           forward=False, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=3,
          n_jobs=1)

x = X_train.iloc[:,:-1].values
y = y_train.iloc[:].values

sbs = sbs.fit(x,y)
sbs.subsets_


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    2.9s finished

[2022-11-02 00:31:20] Features: 69/10 -- score: 0.9064457283191548[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:    2.3s finished

[2022-11-02 00:31:22] Features: 68/10 -- score: 0.9081661498776491[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed:    2.3s finished

[2022-11-02 00:31:25] Features: 67/10 -- score: 0.9115937419941872[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

{70: {'feature_idx': (0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69),
  'cv_scores': array([0.89717224, 0.90463918, 0.89948454]),
  'avg_score': 0.9004319826146875,
  'feature_names': ('0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '24',
   '25',
   '26',
   '27',
   '28',
   '29',
   '30',
   '31',
   '32',
   '33',
   '34',
   '35',
   '36',
   '37',
   '38',
   '39',
   '40',
   '41',
   '42',
   '43',
  

In [10]:
#Sequential backward search result
print(sbs.k_feature_names_)
print(sbs.k_score_)


('2', '3', '4', '8', '9', '13', '14', '15', '40', '67')
0.9072960008480639


In [11]:
!pip install sklearn-genetic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 42.7 MB/s 


In [12]:
#GENETIC ALGORITHM FOR FEATURE SELECTION ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV

x = X_train.iloc[:,:-1].values
y = y_train.iloc[:].values

clf = DecisionTreeClassifier()
model = GeneticSelectionCV(
    clf, cv=5, verbose=0,
    scoring="accuracy", max_features=10,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x, y)

In [13]:
#Do feature selection using the genetic algo
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())

#save file
export_csv= features_df_new_ga.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/features_df_new_ga.csv')

[False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False  True False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False]

 cols =  [13 33 37 67] 

(1165, 4)

 features_df_new_ga=        Intl Calls  State_KY  State_ME  Int'l Plan_no
0              3         0         0              1
1              3         0         0              1
2              5         0         0              1
3              7         0         0              0
4              3         0         0              0
...          ...       ...       ...            ...
1160           2         0         0              1
1161           9         0         0              0
1162           2         0         0          

In [28]:
#K-BEST FEATURE Selection =================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=10)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_train, y_train)
feature_index = fs.get_support(True)
print ("feature index =", feature_index)

#f_value, p_value = f_classif(X_train, y_train)
#print (feature_index.tolist(), f_value, p_value) 

feature index = [ 3  5  6  8 12 14 15 67 68 69]


In [15]:
#Feature selection-Continued
fs = SelectKBest(f_classif, k=10)
new_X_train = fs.fit_transform(X_train,y_train)
print(new_X_train.shape)

new_X_train_SelectedFeatures= pd.DataFrame(new_X_train)
print(new_X_train_SelectedFeatures)

export_csv= new_X_train_SelectedFeatures.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/new_X_train_Selected-Features-Selected Features.csv')


(1165, 10)
          0      1      2      3     4     5    6    7    8    9
0     265.1  45.07  197.4  16.78  10.0  2.70  1.0  1.0  0.0  0.0
1     161.6  27.47  195.5  16.62  13.7  3.70  1.0  1.0  0.0  0.0
2     243.4  41.38  121.2  10.30  12.2  3.29  0.0  1.0  0.0  1.0
3     299.4  50.90   61.9   5.26   6.6  1.78  2.0  0.0  1.0  1.0
4     166.7  28.34  148.3  12.61  10.1  2.73  3.0  0.0  1.0  1.0
...     ...    ...    ...    ...   ...   ...  ...  ...  ...  ...
1160  112.2  19.07  154.8  13.16  10.6  2.86  0.0  1.0  0.0  0.0
1161  170.7  29.02  179.1  15.22   8.2  2.21  3.0  0.0  1.0  1.0
1162  172.5  29.33  142.6  12.12  10.9  2.94  0.0  1.0  0.0  1.0
1163  194.3  33.03  243.2  20.67   7.1  1.92  1.0  1.0  0.0  0.0
1164  187.8  31.93  129.8  11.03  12.0  3.24  2.0  1.0  0.0  0.0

[1165 rows x 10 columns]


In [16]:
#Apply feature selection to test data also

new_X_test = fs.transform(X_test)
print(new_X_test.shape)

new_X_test_SelectedFeatures= pd.DataFrame(new_X_test)
print(new_X_test_SelectedFeatures)

export_csv= new_X_test_SelectedFeatures.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/new_X_test_Selected-Features-Selected Features.csv')


(1261, 10)
          0      1      2      3     4     5    6    7    8    9
0     307.1  52.21  289.4  24.60   8.0  2.16  0.0  0.0  1.0  0.0
1     118.2  20.09  167.2  14.21  12.2  3.29  3.0  1.0  0.0  1.0
2     154.0  26.18  205.9  17.50  12.9  3.48  1.0  1.0  0.0  1.0
3     155.5  26.44  213.4  18.14   7.6  2.05  1.0  1.0  0.0  1.0
4     125.6  21.35  213.0  18.11   5.4  1.46  1.0  0.0  1.0  1.0
...     ...    ...    ...    ...   ...   ...  ...  ...  ...  ...
1256  197.8  33.63  133.9  11.38  10.9  2.94  4.0  1.0  0.0  1.0
1257  115.4  19.62  166.2  14.13  15.2  4.10  2.0  1.0  0.0  0.0
1258  123.8  21.05  236.2  20.08   3.7  1.00  0.0  1.0  0.0  1.0
1259  117.6  19.99  196.3  16.69   6.4  1.73  1.0  1.0  0.0  0.0
1260  105.3  17.90  197.4  16.78   8.6  2.32  2.0  1.0  0.0  0.0

[1261 rows x 10 columns]


In [17]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(new_X_train, y_train)
clf_predict=clf.predict(new_X_test)
print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(new_X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(new_X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(new_X_train,y_train)
clf_predict = clf.predict(new_X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(new_X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))
clf_cv_score = cross_val_score(clf, new_X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


accuracy Score (training) for Decision TreE:0.891356
Confusion Matrix for Decision Tree
[[1020   49]
 [  88  104]]
{'min_samples_split': 30, 'max_depth': 15}
accuracy Score (training) after hypertuning for Decision Tree:0.911182
Confusion Matrix after hypertuning for Decision Tree
[[1056   13]
 [  99   93]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      1069
           1       0.88      0.48      0.62       192

    accuracy                           0.91      1261
   macro avg       0.90      0.74      0.79      1261
weighted avg       0.91      0.91      0.90      1261

=== All AUC Scores ===
[0.78071895 0.91045752 0.82222222 0.7872549  0.82222222 0.87640056
 0.85434174 0.8855042  0.88130252 0.77953795]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.8399962790396687


In [18]:
#CONSTRUCT DEFAULT RANDOM FOREST AND OBTAIN RESPECTIVE ACCURACY 
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))


print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

accuracy Score (training) for RandomForest:0.908803
Confusion Matrix for Random Forest:
[[1068    1]
 [ 114   78]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1069
           1       0.99      0.41      0.58       192

    accuracy                           0.91      1261
   macro avg       0.95      0.70      0.76      1261
weighted avg       0.92      0.91      0.89      1261

=== All AUC Scores ===
[0.81764706 0.93169935 0.89901961 0.80849673 0.9124183  0.94152661
 0.97794118 0.8714986  0.93242297 0.82277228]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8915442678721653


In [19]:
#CONSTRUCT DEFAULT RANDOM FOREST AND OBTAIN RESPECTIVE ACCURACY 
rfc = RandomForestClassifier()
rfc.fit(new_X_train, y_train)
rfc_predict=rfc.predict(new_X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(new_X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(new_X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(new_X_train,y_train)
rfc_predict = rfc.predict(new_X_test)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(new_X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, new_X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

accuracy Score (training) for RandomForest:0.921491
Confusion Matrix for Random Forest:
[[1055   14]
 [  85  107]]
{'min_samples_split': 20, 'max_depth': 7}
accuracy Score (training) after hypertuning for Random Forest:0.906423
Confusion Matrix after hypertuning for Random Forest:
[[1056   13]
 [ 105   87]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      1069
           1       0.87      0.45      0.60       192

    accuracy                           0.91      1261
   macro avg       0.89      0.72      0.77      1261
weighted avg       0.90      0.91      0.89      1261

=== All AUC Scores ===
[0.86666667 0.94052288 0.85882353 0.82156863 0.88823529 0.92717087
 0.96078431 0.8452381  0.92156863 0.8349835 ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8865562396575793


In [20]:
#CONSTRUCT DEFAULT GRADIENT BOOSTING AND OBTAIN RESPECTIVE ACCURACY 

search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(new_X_train, y_train)
abc_predict=abc.predict(new_X_test)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(new_X_test,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(new_X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(new_X_train,y_train)
abc_predict = abc.predict(new_X_test)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(new_X_test,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))
abc_cv_score = cross_val_score(abc, new_X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

accuracy Score (training) for Boosting:0.921491
Confusion Matrix for boosting:
[[1053   16]
 [  83  109]]
{'n_estimators': 20, 'learning_rate': 0.1}
accuracy Score (training) after hypertuning for Boosting:0.900872
Confusion Matrix after hypertuning for Boosting:
[[1062    7]
 [ 118   74]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      1069
           1       0.91      0.39      0.54       192

    accuracy                           0.90      1261
   macro avg       0.91      0.69      0.74      1261
weighted avg       0.90      0.90      0.88      1261

=== All AUC Scores ===
[0.8375817  0.9251634  0.89346405 0.81960784 0.90620915 0.93522409
 0.96813725 0.8627451  0.92892157 0.85082508]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.8927879237503584


# ***SMOTE***

In [22]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
#sm = SMOTE(sampling_strategy='float', ratio=0.5)
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(new_X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 1019, 1: 146})
Resampled dataset shape Counter({0: 1019, 1: 509})


# ***ENSEMBLE METHODS STACKING***

In [23]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           new_X_train, y_train, new_X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.90410959]
    fold  1:  [0.93814433]
    fold  2:  [0.92096220]
    fold  3:  [0.88659794]
    ----
    MEAN:     [0.91245351] + [0.01917386]
    FULL:     [0.91244635]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91095890]
    fold  1:  [0.93814433]
    fold  2:  [0.92096220]
    fold  3:  [0.90034364]
    ----
    MEAN:     [0.91760227] + [0.01392174]
    FULL:     [0.91759657]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.86301370]
    fold  1:  [0.92096220]
    fold  2:  [0.91408935]
    fold  3:  [0.85567010]
    ----
    MEAN:     [0.88843384] + [0.02930847]
    FULL:     [0.88841202]



In [24]:
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_train)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [0.90483743]


In [25]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.180741,0.819259
1,0.946463,0.053537
2,0.946463,0.053537
3,0.946463,0.053537
4,0.946463,0.053537
